In [14]:
from solve import search

In [15]:
with open('beispieldaten/kaese8.txt', 'r') as f:
    stack = [tuple(map(int, f.readline().split())) for _ in range(int(f.readline()))]

In [16]:
for path, size, n_virtual, n_nodes in search(stack):
    print(f'{path=}, {size=}, {n_virtual=}, {n_nodes=}')
# a combination of multiple of these partial solutions should (only nodes in the end can be removed) use up all slices
# todo maybe disallow permutations of the same slice
# 'virtual' slices are included in the path, but not counted towards the number of elements and always added/removed in conjunction with a real next slice
# for a given size choose the one with the least virtual slices (maybe bfs)

path=(0,), size=(1, 2, 1), n_virtual=0, n_nodes=1
path=(1, 0, (1, 2), 2), size=(2, 2, 5), n_virtual=1, n_nodes=3
path=(2, 0), size=(1, 2, 6), n_virtual=0, n_nodes=2


In [17]:
from typing import Generator, List, Tuple


def combinations(l: List, target_length: int) -> Generator[List, None, None]:
    """Yield all combinations of the elements in l."""
    l.sort(key=lambda x: x[3])    # sort by node length

    if len(l) == 0:
        yield []
    else:
        for i in range(len(l)):
            for c in combinations(l[:i] + l[i + 1 :], target_length - l[i][3]):
                if len(c) == 1 and c[0][3] != target_length:
                    continue
                yield [l[i]] + c

In [18]:
for a in search(stack):
    print(a)

((0,), (1, 2, 1), 0, 1)
((1, 0, (1, 2), 2), (2, 2, 5), 1, 3)
((2, 0), (1, 2, 6), 0, 2)


In [19]:
# TODO onyl go upward from first to avoid duplicate paths, ability to remove nodes from the end
import itertools

def remove(slice: Tuple[int, int], size: List[int]):
    ab = set(slice)
    if ab == set(size[:2]):
        size[2] -= 1
    elif ab == set(size[1:]):
        size[0] -= 1
    elif ab == set(size[::2]):
        size[1] -= 1
    else:
        raise Exception('what')

for l in range(1, 3):
    for c in itertools.combinations(search(stack), l):
        overflow = sum(x[3] for x in c) - len(stack)
        if overflow < 0:
            continue
        for x in itertools.product(range(overflow + 1), repeat=l):
            if sum(x) != overflow:
                continue
            paths = []
            for i, p in enumerate(c):
                path, size, n_virtual = list(p[0]), list(p[1]), p[2]
                for _ in range(x[i]):
                    i = path.pop()
                    remove(stack[i], size)
                    if path and isinstance(path[-1], tuple):
                        s = path.pop()
                        remove(s, size)
                        n_virtual -= 1

                paths.append((tuple(path), tuple(size), n_virtual))
            if any(len(p) == 0 for p, _, _ in paths):
                continue
            seen = set()
            for p, _, _ in paths:
                for i in p:
                    if isinstance(i, tuple):
                        continue
                    seen.add(i)
            if len(seen) == len(stack):
                print(paths, len(paths))
        

[((1, 0, (1, 2), 2), (2, 2, 5), 1)] 1
[((1, 0), (1, 2, 4), 0), ((2,), (1, 2, 5), 0)] 2
[((1,), (1, 2, 3), 0), ((2, 0), (1, 2, 6), 0)] 2


In [20]:
# initialpath + rest(stack - initialpath) -> unpath (gets removed from the end of initial path)
# nodes: set of nodes available, stock: list of previous solution in order of possible removal -> list of solutions accepted
# len of stock is len(stack) - len(nodes)
def make_path(nodes: set, stock: list or None) -> list(path):
    pass

In [24]:
from solve import make_stacks

for paths in make_stacks(stack, 2):
    print('next paths')
    for path in paths:
        path_, size, n_virtual = path
        print(f'{path_=}, {size=}, {n_virtual=}')

next paths
path_=(1, 0), size=(1, 2, 4), n_virtual=0
path_=(2,), size=(1, 2, 5), n_virtual=0
next paths
path_=(1,), size=(1, 2, 3), n_virtual=0
path_=(2, 0), size=(1, 2, 6), n_virtual=0
